# TNT Sticky Rouse: Stress Relaxation

## Objectives
- Fit TNT Sticky Rouse model to stress relaxation data
- Analyze multi-exponential relaxation spectrum
- Understand sticker-truncated spectrum and plateau formation
- Demonstrate mode-resolved relaxation dynamics

## Setup

In [1]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTStickyRouse

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_laponite_relaxation,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_sticky_rouse_param_names,
    plot_sticky_rouse_effective_times,
    plot_mode_decomposition,
)

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

print("Setup complete. JAX devices:", jax.devices())

Setup complete. JAX devices: [CpuDevice(id=0)]


## Theory: Multi-Exponential Relaxation

**Relaxation Modulus:**

$$G(t) = \sum_k G_k \exp\left(-\frac{t}{\tau_{\textrm{eff},k}}\right)$$

where τ_eff,k = max(τ_R,k, τ_s)

**Sticker-Truncated Spectrum:**
- Modes with τ_R,k < τ_s: All relax on timescale τ_s → plateau in G(t)
- Modes with τ_R,k > τ_s: Individual exponential decay

**Key Physics:**
- **Early times (t ≪ τ_s)**: All modes frozen → G(t) ≈ G_total = Σ G_k
- **Intermediate times (t ~ τ_s)**: Sticker-dominated modes relax collectively → plateau
- **Late times (t ≫ τ_s)**: Rouse-dominated modes decay individually

**Observable Signatures:**
- Initial elastic modulus: G(0) = Σ G_k
- Sticker plateau: G_plateau = Σ G_k (for modes with τ_R,k < τ_s)
- Long-time decay: Determined by slowest Rouse mode

## Load Data

In [2]:
# Load Laponite relaxation data (aged 1800s = 30 min)
time_data, G_t = load_laponite_relaxation(aging_time=1800)

print(f"Data shape: {len(time_data)} points")
print(f"Time range: {time_data.min():.2e} - {time_data.max():.2e} s")
print(f"G(t) range: {G_t.min():.2e} - {G_t.max():.2e} Pa")
print(f"Aging time: 1800 s (30 minutes)")

# Plot raw data
fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(time_data, G_t, 'ko', label='Laponite data', markersize=6)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('G(t) (Pa)', fontsize=12)
ax.set_title('Stress Relaxation: Laponite Gel', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.close("all")
plt.close('all')

Data shape: 27 points
Time range: 3.50e-01 - 1.73e+03 s
G(t) range: 2.24e+02 - 4.60e+02 Pa
Aging time: 1800 s (30 minutes)


## NLSQ Fitting

In [3]:
# Initialize model
model = TNTStickyRouse(n_modes=3)
param_names = get_tnt_sticky_rouse_param_names(n_modes=3)
print(f"Model parameters ({len(param_names)}): {param_names}")

# Fit using NLSQ
print("\nFitting with NLSQ...")
start_time = time.time()
model.fit(time_data, G_t, test_mode="relaxation", method='scipy')
fit_time = time.time() - start_time

# Compute metrics
G_pred_train = model.predict(time_data, test_mode="relaxation")
metrics_nlsq = compute_fit_quality(G_t, G_pred_train)

print(f"\nFit completed in {fit_time:.2f} seconds")
print(f"R² = {metrics_nlsq['R2']:.6f}")
print(f"RMSE = {metrics_nlsq['RMSE']:.4e} Pa")

INFO:nlsq.curve_fit:Starting curve fit n_params=8 | n_data_points=27 | method=trf | solver=auto | batch_size=None | has_bounds=True | dynamic_sizing=False


INFO:nlsq.least_squares:Starting least squares optimization method=trf | n_params=8 | loss=linear | ftol=1.0000e-08 | xtol=1.0000e-08 | gtol=1.0000e-08


Model parameters (8): ['G_0', 'tau_R_0', 'G_1', 'tau_R_1', 'G_2', 'tau_R_2', 'tau_s', 'eta_s']

Fitting with NLSQ...


PERFORMANCE:nlsq.least_squares:Timer: optimization elapsed=1.797040s


INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=20 | final_cost=1.1694e+06 | elapsed=1.797s | final_gradient_norm=7.3105e-07


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit elapsed=2.216879s


INFO:nlsq.curve_fit:Curve fit completed total_time=2.2169 | final_cost=2.3388e+06 | covariance_warning=False


ERROR:rheojax.core.base:Predict failed
Traceback (most recent call last):
  File "/Users/b80985/Projects/rheojax/rheojax/core/base.py", line 713, in predict
    result = self._predict(X, **kwargs)
  File "/Users/b80985/Projects/rheojax/rheojax/models/tnt/sticky_rouse.py", line 550, in _predict
    raise ValueError("test_mode must be specified for prediction")
ValueError: test_mode must be specified for prediction



Fit completed in 2.37 seconds
R² = -41.292469
RMSE = 3.2198e+02 Pa


## Fitted Parameters

In [4]:
# Extract fitted parameters
params_nlsq = {name: model.parameters.get_value(name) for name in param_names}

print("\nFitted Parameters:")
print("-" * 50)
for name, value in params_nlsq.items():
    if 'tau' in name:
        print(f"{name:10s} = {value:12.4e} s")
    elif 'eta' in name:
        print(f"{name:10s} = {value:12.4e} Pa·s")
    else:
        print(f"{name:10s} = {value:12.4e} Pa")

# Compute total modulus
G_total = sum(params_nlsq[f'G_{i}'] for i in range(3))
print(f"\nTotal elastic modulus: G(0) = {G_total:.4e} Pa")

# Analyze effective relaxation times
tau_s = params_nlsq['tau_s']
print(f"Sticker lifetime: τ_s = {tau_s:.4e} s")
print("\nRelaxation Spectrum:")
print("-" * 50)
for i in range(3):
    tau_R = params_nlsq[f'tau_R_{i}']
    tau_eff = max(tau_R, tau_s)
    G_i = params_nlsq[f'G_{i}']
    regime = "STICKER-TRUNCATED" if tau_s > tau_R else "ROUSE"
    print(f"Mode {i}: G = {G_i:.3e} Pa, τ_eff = {tau_eff:.3e} s ({regime})")


Fitted Parameters:
--------------------------------------------------
G_0        =   5.8967e+00 Pa
tau_R_0    =   2.6954e-02 s
G_1        =   1.4623e+01 Pa
tau_R_1    =   3.7516e-03 s
G_2        =   1.1819e+01 Pa
tau_R_2    =   5.8685e-04 s
tau_s      =   3.6359e-03 s
eta_s      =   2.5297e-05 Pa·s

Total elastic modulus: G(0) = 3.2339e+01 Pa
Sticker lifetime: τ_s = 3.6359e-03 s

Relaxation Spectrum:
--------------------------------------------------
Mode 0: G = 5.897e+00 Pa, τ_eff = 2.695e-02 s (ROUSE)
Mode 1: G = 1.462e+01 Pa, τ_eff = 3.752e-03 s (ROUSE)
Mode 2: G = 1.182e+01 Pa, τ_eff = 3.636e-03 s (STICKER-TRUNCATED)


## NLSQ Prediction vs Data

In [5]:
# Get gamma_dot for relaxation protocol (pre-shear rate)
gamma_dot = 10.0  # s^-1, typical pre-shear rate

# Compute metrics for plot title
metrics = compute_fit_quality(G_t, model.predict(time_data, test_mode='relaxation', gamma_dot=gamma_dot))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    time_data, G_t, model, test_mode="relaxation",
    param_names=param_names, log_scale=True,
    xlabel='Time (s)',
    ylabel=r'Relaxation modulus $G(t)$ (Pa)',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})',
    gamma_dot=gamma_dot
)
plt.close("all")

## Multi-Exponential Decomposition

In [6]:
time_fine = np.linspace(time_data.min(), time_data.max(), 200)
G_pred = model.predict(time_fine, test_mode='relaxation')

# Plot individual mode contributions
fig, ax = plt.subplots(figsize=(10, 7))

# Total prediction
ax.loglog(time_fine, G_pred, 'k-', label='Total', linewidth=2.5, zorder=5)

# Individual modes
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
for i in range(3):
    G_i = params_nlsq[f'G_{i}']
    tau_R_i = params_nlsq[f'tau_R_{i}']
    tau_eff_i = max(tau_R_i, tau_s)
    
    G_i_t = G_i * np.exp(-time_fine / tau_eff_i)
    
    regime = "sticker" if tau_s > tau_R_i else "Rouse"
    ax.loglog(time_fine, G_i_t, '--', color=colors[i], 
             label=f'Mode {i} ({regime}, τ_eff={tau_eff_i:.2e}s)', linewidth=1.5)

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('G(t) (Pa)', fontsize=12)
ax.set_title('Multi-Exponential Decomposition', fontsize=14, fontweight='bold')
ax.legend(fontsize=9, loc='best')
ax.grid(True, alpha=0.3)
plt.close("all")
plt.close('all')

## Sticker-Truncated Spectrum Visualization

In [7]:
# Analyze sticker plateau formation
tau_s = params_nlsq['tau_s']
n_sticker_modes = sum(1 for i in range(3) if params_nlsq[f'tau_R_{i}'] < tau_s)

print(f"Sticker-truncated modes: {n_sticker_modes}/3")

if n_sticker_modes > 0:
    # Sticker plateau modulus
    G_plateau = sum(params_nlsq[f'G_{i}'] for i in range(3) if params_nlsq[f'tau_R_{i}'] < tau_s)
    
    # Compute G(t) with and without sticker truncation
    G_without_sticker = sum(params_nlsq[f'G_{i}'] * np.exp(-time_fine / params_nlsq[f'tau_R_{i}']) 
                            for i in range(3))
    
    fig, ax = plt.subplots(figsize=(10, 7))
    ax.loglog(time_data, G_t, 'ko', label='Data', markersize=6, alpha=0.5)
    ax.loglog(time_fine, G_pred, 'b-', label='With Stickers (Fit)', linewidth=2)
    ax.loglog(time_fine, G_without_sticker, 'r--', label='Without Stickers (Rouse only)', linewidth=2)
    
    # Highlight sticker plateau region
    ax.axhline(G_plateau, color='green', linestyle=':', linewidth=1.5, 
               label=f'Sticker Plateau (G_s={G_plateau:.2e} Pa)')
    ax.axvline(tau_s, color='purple', linestyle=':', linewidth=1.5, 
               label=f'Sticker timescale (τ_s={tau_s:.2e}s)')
    
    ax.set_xlabel('Time (s)', fontsize=12)
    ax.set_ylabel('G(t) (Pa)', fontsize=12)
    ax.set_title('Sticker-Truncated vs Pure Rouse Relaxation', fontsize=14, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    plt.close("all")
    plt.close('all')
    
    print(f"\nSticker plateau modulus: G_plateau = {G_plateau:.4e} Pa")
    print(f"Plateau fraction: G_plateau/G_total = {G_plateau/G_total:.2%}")
else:
    print("\nNo sticker-truncated modes detected. All modes exhibit intrinsic Rouse relaxation.")

Sticker-truncated modes: 1/3

Sticker plateau modulus: G_plateau = 1.1819e+01 Pa
Plateau fraction: G_plateau/G_total = 36.55%


## Effective Relaxation Time Analysis

In [8]:
# Visualize sticker-mode interaction
fig = plot_sticky_rouse_effective_times(model)
plt.close("all")
plt.close('all')

## Bayesian Inference

In [9]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    result_bayes = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    result_bayes = model.fit_bayesian(
        time_data, G_t,
        test_mode='relaxation',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)
To run Bayesian analysis, run with FAST_MODE=0


## Convergence Diagnostics

In [10]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(result_bayes, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


FAST_MODE: Skipping convergence diagnostics


## Parameter Comparison: NLSQ vs Bayesian

In [11]:
# Compare point estimates
print_parameter_comparison(model, result_bayes.posterior_samples, param_names)


Parameter Comparison: NLSQ vs Bayesian
      Parameter          NLSQ        Median                          95% CI
---------------------------------------------------------------------------
            G_0         5.897         5.897  [5.897, 5.897]
        tau_R_0       0.02695       0.02695  [0.02695, 0.02695]
            G_1         14.62         14.62  [14.62, 14.62]
        tau_R_1      0.003752      0.003752  [0.003752, 0.003752]
            G_2         11.82         11.82  [11.82, 11.82]
        tau_R_2     0.0005868     0.0005868  [0.0005868, 0.0005868]
          tau_s      0.003636      0.003636  [0.003636, 0.003636]
          eta_s      2.53e-05      2.53e-05  [2.53e-05, 2.53e-05]


## ArviZ Diagnostics

In [12]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(result_bayes, 'to_inference_data'):
    display_arviz_diagnostics(result_bayes, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

FAST_MODE: Skipping ArviZ diagnostics


## Posterior Predictive Distribution

In [13]:
# Posterior predictive check
if not FAST_MODE and hasattr(result_bayes, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        time_data, G_t, model, result_bayes,
        test_mode="relaxation", param_names=param_names,
        log_scale=True,
        xlabel=r'Time (s)',
        ylabel=r'Relaxation modulus $G(t)$ (Pa)',
        gamma_dot=gamma_dot
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

FAST_MODE: Skipping posterior predictive


## Physical Interpretation

In [14]:
# Extract posterior means
posterior = result_bayes.posterior_samples
params_bayes = {name: float(np.mean(posterior[name])) for name in param_names}
tau_s_bayes = params_bayes['tau_s']

print("Physical Interpretation (Posterior Means):")
print("=" * 60)

# Total modulus
G_total_bayes = sum(params_bayes[f'G_{i}'] for i in range(3))
print(f"\nInitial Elastic Modulus: G(0) = {G_total_bayes:.4e} Pa")
print(f"Sticker Lifetime: τ_s = {tau_s_bayes:.4e} s")

print("\nRelaxation Spectrum Analysis:")
print("-" * 60)
for i in range(3):
    G_i = params_bayes[f'G_{i}']
    tau_R_i = params_bayes[f'tau_R_{i}']
    tau_eff_i = max(tau_R_i, tau_s_bayes)
    weight = G_i / G_total_bayes
    
    print(f"\nMode {i}:")
    print(f"  Modulus: G_{i} = {G_i:.4e} Pa ({weight:.1%} of total)")
    print(f"  Rouse time: τ_R,{i} = {tau_R_i:.4e} s")
    print(f"  Effective time: τ_eff,{i} = {tau_eff_i:.4e} s")
    
    if tau_s_bayes > tau_R_i:
        print(f"  ✓ STICKER-TRUNCATED: Relaxation limited by sticker lifetime")
        print(f"    Without stickers would relax {tau_s_bayes/tau_R_i:.1f}x faster")
    else:
        print(f"  ✓ ROUSE RELAXATION: Intrinsic chain dynamics dominate")
        print(f"    Stickers dissolve {tau_R_i/tau_s_bayes:.1f}x faster than chain relaxes")

# Sticker plateau analysis
n_sticker_modes = sum(1 for i in range(3) if params_bayes[f'tau_R_{i}'] < tau_s_bayes)
if n_sticker_modes > 0:
    G_plateau = sum(params_bayes[f'G_{i}'] for i in range(3) if params_bayes[f'tau_R_{i}'] < tau_s_bayes)
    print(f"\nSticker Plateau:")
    print(f"  Number of truncated modes: {n_sticker_modes}/3")
    print(f"  Plateau modulus: G_plateau = {G_plateau:.4e} Pa")
    print(f"  Plateau fraction: {G_plateau/G_total_bayes:.1%} of total modulus")
    print(f"  Plateau lifetime: τ_s = {tau_s_bayes:.4e} s")

# Longest relaxation time
tau_max = max(max(params_bayes[f'tau_R_{i}'], tau_s_bayes) for i in range(3))
print(f"\nLongest Relaxation Time: τ_max = {tau_max:.4e} s")
print(f"Terminal relaxation: G(t→∞) decays as exp(-t/{tau_max:.2e})")

Physical Interpretation (Posterior Means):

Initial Elastic Modulus: G(0) = 3.2339e+01 Pa
Sticker Lifetime: τ_s = 3.6359e-03 s

Relaxation Spectrum Analysis:
------------------------------------------------------------

Mode 0:
  Modulus: G_0 = 5.8967e+00 Pa (18.2% of total)
  Rouse time: τ_R,0 = 2.6954e-02 s
  Effective time: τ_eff,0 = 2.6954e-02 s
  ✓ ROUSE RELAXATION: Intrinsic chain dynamics dominate
    Stickers dissolve 7.4x faster than chain relaxes

Mode 1:
  Modulus: G_1 = 1.4623e+01 Pa (45.2% of total)
  Rouse time: τ_R,1 = 3.7516e-03 s
  Effective time: τ_eff,1 = 3.7516e-03 s
  ✓ ROUSE RELAXATION: Intrinsic chain dynamics dominate
    Stickers dissolve 1.0x faster than chain relaxes

Mode 2:
  Modulus: G_2 = 1.1819e+01 Pa (36.5% of total)
  Rouse time: τ_R,2 = 5.8685e-04 s
  Effective time: τ_eff,2 = 3.6359e-03 s
  ✓ STICKER-TRUNCATED: Relaxation limited by sticker lifetime
    Without stickers would relax 6.2x faster

Sticker Plateau:
  Number of truncated modes: 1/3
  Plat

## Save Results

In [15]:
# Save results to disk
output_path = save_tnt_results(model, result_bayes, "sticky_rouse", "relaxation", param_names)
print(f"Results saved to: {output_path}")

Results saved to /Users/b80985/Projects/rheojax/examples/tnt/../utils/../outputs/tnt/sticky_rouse/relaxation/
  nlsq_params_relaxation.json: 8 parameters
  posterior_relaxation.json: 500 draws
Results saved to: None


## Key Takeaways

1. **Multi-Exponential Nature**: Relaxation modulus is sum of exponentials with distinct timescales

2. **Sticker Truncation**: Fast Rouse modes (τ_R,k < τ_s) are limited by sticker lifetime → collective decay

3. **Plateau Formation**: Modes with τ_R < τ_s create a plateau at intermediate times in G(t)

4. **Spectrum Modification**: Stickers fundamentally alter the relaxation spectrum by truncating fast processes

5. **Timescale Hierarchy**: The effective relaxation spectrum reveals which modes are sticker-limited vs Rouse-limited

6. **Bayesian Insights**: Posterior uncertainty quantifies confidence in mode assignment and spectrum structure